In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from sklearn import preprocessing
import random
import tensorflow as tf

In [ ]:
len(os.listdir("../input/ocular-disease-recognition-odir5k/preprocessed_images"))

In [ ]:
df = pd.read_csv("../input/ocular-disease-recognition-odir5k/full_df.csv")
df.head()

In [ ]:
count = 1
f = plt.figure(figsize=(50,20))
for Class in df['labels'].unique():
    seg = df[df['labels']==Class]
    address = seg.sample().iloc[0]['filename']
    dataset_dir = "/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images/"
    img = cv2.imread(dataset_dir+ address)
    #print(img)
    ax = f.add_subplot(2, 4,count)
    ax = plt.imshow(img)
    ax = plt.title(Class,fontsize= 30)
    count = count + 1
plt.show()

In [ ]:
def has_myopia(text):
    if "myopia" in text:
        return 1
    else:
        return 0

In [ ]:
df["left_myopia"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_myopia(x))
df["right_myopia"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_myopia(x))

In [ ]:
left_myopia_imgs = []
right_myopia_imgs = []
for i in range(len(df)):
    if df["left_myopia"][i] == 1:
        left_myopia_imgs.append(df['Left-Fundus'][i])
    if df["right_myopia"][i] == 1:
        right_myopia_imgs.append(df['Right-Fundus'][i])

In [ ]:
print(len(left_myopia_imgs))
print(len(right_myopia_imgs))

In [ ]:
def is_normal(text):
    if "normal fundus" in text:
        return 1
    else:
        return 0

In [ ]:
df["left_normal"] = df["Left-Diagnostic Keywords"].apply(lambda x: is_normal(x))
df["right_normal"] = df["Right-Diagnostic Keywords"].apply(lambda x: is_normal(x))

In [ ]:
left_normal_imgs = []
right_normal_imgs = []
for i in range(len(df)):
    if df["left_normal"][i] == 1:
        left_normal_imgs.append(df['Left-Fundus'][i])
    if df["right_normal"][i] == 1:
        right_normal_imgs.append(df['Right-Fundus'][i])

In [ ]:
print(len(left_normal_imgs))
print(len(right_normal_imgs))

In [ ]:
import random
left_normal_imgs = random.sample(left_normal_imgs, 304)
right_normal_imgs = random.sample(right_normal_imgs, 290)

In [ ]:
print(len(left_normal_imgs))
print(len(right_normal_imgs))

In [ ]:
myopia = np.concatenate((left_myopia_imgs,right_myopia_imgs),axis=0)
normal = np.concatenate((left_normal_imgs,right_normal_imgs),axis=0)

In [ ]:
print(len(myopia),len(normal))

In [ ]:
images_dir = "../input/ocular-disease-recognition-odir5k/preprocessed_images/"
image_size=224
labels = []
dataset = []
def create_dataset(image_category,label):
    for img in image_category:
        image_path = os.path.join(images_dir,img)
        try:
            image = cv2.imread(image_path)
            image = cv2.resize(image,(image_size,image_size))
        except:
            continue
        
        dataset.append([np.array(image),np.array(label)])
    random.shuffle(dataset)
    return dataset

dataset = create_dataset(myopia,1)
dataset = create_dataset(normal,0)

In [ ]:
len(dataset)

In [ ]:
X = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
Y = np.array([i[1] for i in dataset])

In [ ]:
X.shape, Y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [ ]:
X_val = X_train[-50:]
Y_val = Y_train[-50:]
X_train = X_train[:-50]
Y_train = Y_train[:-50]

In [ ]:
print(f"X_train Shape: {X_train.shape}, Y_train Shape: {Y_train.shape}")
print(f"X_val Shape: {X_val.shape}, Y_val Shape: {Y_val.shape}")
print(f"X_test Shape: {X_test.shape}, Y_test Shape: {Y_test.shape}")

In [ ]:
from tensorflow.keras.applications import ResNet50
resnet = ResNet50(weights="imagenet", include_top = False, input_shape=(image_size,image_size,3))

In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense

model = Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss",patience=7, verbose=1)
callbacks = [early_stopping]

In [ ]:
hist = model.fit(X_train, Y_train, batch_size=8, epochs=100, validation_data=(X_val, Y_val),
                    verbose=1,callbacks=callbacks)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
model.evaluate(X_test, Y_test, batch_size=8)

In [ ]:
y_pred = model.predict(X_test, batch_size=8)
y_pred[y_pred <= 0.5] = 0.
y_pred[y_pred > 0.5] = 1.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(Y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))